In [6]:
# Testing the addition of NCMC moves in saltwap.
# Importing:
from datetime import datetime
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import sys
import gc
sys.path.append("../saltswap/")

import saltswap

# Defining constants.
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure = 1*unit.atmospheres
temperature = 300*unit.kelvin
delta_chem = +650*unit.kilojoule_per_mole  # should be around -630 kJ/mol for bulk water with 0.2 M salt.

In [7]:
#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

In [8]:
# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

In [10]:
# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 10           # Number of NCMC steps
nverlet = 1            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=False,nkernals=nkernals, nverlet_steps=nverlet)

In [11]:
# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

In [12]:
# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

In [13]:
mc_saltswap.getIdentityCounts()

(501, 0, 0)

In [14]:
# Cycles of MD and MC salt exchanges:
iterations = 10        # Number of rounds of MD and constant salt moves
nsteps = 200           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 
printfreq = 1

pdbfile = open('output.pdb', 'w')
app.PDBFile.writeHeader(pdb.topology, file=pdbfile)
app.PDBFile.writeModel(pdb.topology, positions, file=pdbfile, modelIndex=0)

startTime = datetime.now()
for i in range(iterations):
    startTime = datetime.now()
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)
    app.PDBFile.writeModel(pdb.topology, positions, file=pdbfile, modelIndex=i+1)
    tm = datetime.now() - startTime
    if i%printfreq == 0:
        print "Step %i complete" % i
        print "Identity counts:", mc_saltswap.getIdentityCounts()
        print "Number of NaNs = ",mc_saltswap.nan
        print "Time in seconds = ",tm.seconds
    gc.collect()    
tm = datetime.now() - startTime

#print "\nSimulation time = ",tm.seconds
print "\nAcceptance probability =",mc_saltswap.getAcceptanceProbability()
print "NaNs = ",mc_saltswap.nan

Step 0 complete
Identity counts: (493, 4, 4)
Number of NaNs =  0
Time in seconds =  1
Step 1 complete
Identity counts: (489, 6, 6)
Number of NaNs =  0
Time in seconds =  1
Step 2 complete
Identity counts: (483, 9, 9)
Number of NaNs =  0
Time in seconds =  1
Step 3 complete
Identity counts: (479, 11, 11)
Number of NaNs =  0
Time in seconds =  1
Step 4 complete
Identity counts: (475, 13, 13)
Number of NaNs =  0
Time in seconds =  1
Step 5 complete
Identity counts: (469, 16, 16)
Number of NaNs =  0
Time in seconds =  1
Step 6 complete
Identity counts: (467, 17, 17)
Number of NaNs =  0
Time in seconds =  1
Step 7 complete
Identity counts: (463, 19, 19)
Number of NaNs =  0
Time in seconds =  1
Step 8 complete
Identity counts: (459, 21, 21)
Number of NaNs =  0
Time in seconds =  1
Step 9 complete
Identity counts: (451, 25, 25)
Number of NaNs =  0
Time in seconds =  1

Acceptance probability = 0.5
NaNs =  0


In [32]:
mc_saltswap.update(context,nattempts=50,cost=+10000)

In [33]:
mc_saltswap.getIdentityCounts()

(453, 24, 24)

In [12]:
mc_saltswap.getIdentityCounts()

(489, 6, 6)

With nkernals = 10, nverlet = 1, nsteps = 250000, nattempts = 10. Iterations prematurely cancelled.

Step 0 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  606
Step 1 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  606
Step 2 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  606
Step 3 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  604
Step 4 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  639
Step 5 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  626
Step 6 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  624
Step 7 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  627
Step 8 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  634
Step 9 complete
Identity counts: (501, 0, 0)
Number of NaNs =  0
Time in seconds =  630

With iterations =100, nsteps = 10, nattampts =5:
- No NCMC:
    Simulation time =  7
    Acceptance probability = 0.016
- nkernals = 2, nverlet = 1:
    Simulation time = 24
    Acceptance probability = 0.09
    - repeat:
        Simulation time = 23
        Acceptance probability = 0.04    
- nkernals = 5, nverlet = 1:
   Simulation blows up?
- nkernals = 5, nverlet = 2:   
    Simulation time = 74
    Acceptance probability = 0.63
    - repeat:
        Simulation time = 74
        Acceptance probability = 0.682
- nkernals = 2, nverlet = 2
    Simulation blows up?
- nkernals = 10, nverlet = 2
    Simulation time =  137
    Acceptance probability = 0.07
    - repeat:
        Simulation time =  141
        Acceptance probability = 0.062
- nkernals = 5, nverlet = 5
    Simulation time =  166
    Acceptance probability = 0.06
- nkernals = 100, nverlet = 10, deltachem=-630
    Simulation time = 627
    Acceptance probability = 0.50
- nkernals = 1000, nverlet = 1, deltachem=-630
    Simulation time = 640
    Acceptance probability = 0.4
    
    
With iterations =200, nsteps = 5000, nattampts =5:
 

Timing script to see where memory leak is occuring

In [15]:
startTime = datetime.now()
integrator.step(nsteps)
tm = datetime.now() - startTime
print "Time in seconds = ",tm.seconds

Time in seconds =  679


This is roughly 11.3 minutes, and is close to the 800 seconds the code took previously before the massive increase in time. The memory leak/delay is probably happening the the update part, most likely something to do with NCMC.

In [10]:
import cProfile

In [23]:
cProfile.run("mc_saltswap.update(context,nattempts=nattempts)")

         20222484 function calls (20222284 primitive calls) in 133.766 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  133.766  133.766 <string>:1(<module>)
       30    0.000    0.000    0.000    0.000 _methods.py:31(_sum)
       16    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
   360790    0.769    0.000    1.485    0.000 copy.py:145(deepcopy)
   360770    0.047    0.000    0.047    0.000 copy.py:198(_deepcopy_atomic)
   360790    0.510    0.000    0.560    0.000 copy.py:267(_keep_alive)
       30    0.000    0.000    0.000    0.000 fromnumeric.py:1737(sum)
       16    0.000    0.000    0.000    0.000 fromnumeric.py:2390(prod)
    10000    0.016    0.000  122.673    0.012 openmm.py:10415(step)
    60060    0.033    0.000    3.662    0.000 openmm.py:14807(setParticleParameters)
    10010    0.007    0.000    0.426    0.000 openmm.py:14976(updateParametersInContext)
       60    0.0

In [24]:
10*100/60

16

In [26]:
cProfile.run("mc_saltswap.update(context,nattempts=nattempts)")

         20222464 function calls (20222264 primitive calls) in 133.976 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  133.976  133.976 <string>:1(<module>)
       30    0.000    0.000    0.000    0.000 _methods.py:31(_sum)
       14    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
   360790    0.769    0.000    1.477    0.000 copy.py:145(deepcopy)
   360770    0.047    0.000    0.047    0.000 copy.py:198(_deepcopy_atomic)
   360790    0.501    0.000    0.552    0.000 copy.py:267(_keep_alive)
       30    0.000    0.000    0.000    0.000 fromnumeric.py:1737(sum)
       14    0.000    0.000    0.000    0.000 fromnumeric.py:2390(prod)
    10000    0.015    0.000  122.941    0.012 openmm.py:10415(step)
    60060    0.032    0.000    3.652    0.000 openmm.py:14807(setParticleParameters)
    10010    0.007    0.000    0.419    0.000 openmm.py:14976(updateParametersInContext)
       60    0.0

In [32]:
print mc_saltswap.getIdentityCounts()
print mc_saltswap.getAcceptanceProbability()
print sys.getsizeof(mc_saltswap)

(397, 52, 52)
0.2
64


In [12]:
cProfile.run("mc_saltswap.update(context,nattempts=1000)")

         29370004 function calls (29350004 primitive calls) in 157.899 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  157.899  157.899 <string>:1(<module>)
     3000    0.002    0.000    0.022    0.000 _methods.py:31(_sum)
     1000    0.001    0.000    0.010    0.000 _methods.py:34(_prod)
   439000    0.929    0.000    1.804    0.000 copy.py:145(deepcopy)
   437000    0.056    0.000    0.056    0.000 copy.py:198(_deepcopy_atomic)
   439000    0.612    0.000    0.674    0.000 copy.py:267(_keep_alive)
     3000    0.006    0.000    0.029    0.000 fromnumeric.py:1737(sum)
     1000    0.006    0.000    0.016    0.000 fromnumeric.py:2390(prod)
    10000    0.017    0.000  121.326    0.012 openmm.py:10415(step)
    66000    0.039    0.000    3.955    0.000 openmm.py:14807(setParticleParameters)
    11000    0.008    0.000    0.449    0.000 openmm.py:14976(updateParametersInContext)
     6000    0.0

In [35]:
print mc_saltswap.getIdentityCounts()
print mc_saltswap.getAcceptanceProbability()
print sys.getsizeof(mc_saltswap)

(453, 24, 24)
0.308108108108
64


In [37]:
t= datetime.now()

In [44]:
t.date()

datetime.date(2016, 6, 6)